In [1]:
try:
    from configuracoes_notebooks import set_proj_dir
    set_proj_dir()
except:
    from teste import diretorio_teste
    diretorio_teste()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)

# Ocorrências em 2024

Estávamos usando as manchas de inundação (2025), mas como se tratava de uma previsão para daqui a 25 anos, não fazia sentido usar. Optaram por substituí-la pelos pontos de ocorrência. 

Mas as informações que vem da malha de pontos de ocorrência de inundação são referentes apenas a janeiro e fevereiro de 2025.

Por conta disso, optamos por utilizar o shapefile das ocorrências da Proteção e Defesa Civil, no ano de 2024, disponível em:  [GEOSAMPA/'Download de arquivos': representado por uma imagem de SP com uma seta pra baixo)/Proteção e Defesa Civil/Ocorrencias](https://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadCamadas&arq=07_Prote%E7%E3o%20e%20Defesa%20Civil%5C%5COcorrencias%5C%5CShapefile%5C%5CSIRGAS_SHP_riscoocorrencia&arqTipo=Shapefile)

In [3]:
data_path = get_data_diretorio()

# Importar dados

In [4]:
gdf_ocorrencias = gpd.read_file(
    os.path.join(
        data_path,
        'manual_download',
        'SIRGAS_SHP_riscoocorrencia_2024'
    )
)

In [5]:
gdf_ocorrencias.sample(2)

,data,ocorrencia,subpreit,name,geometry
2084,2024-11-01,QUEDA DE ARVORE,IQ - ITAQUERA,33640857,POINT (352435.145 7398362.394)
4596,2024-01-28,QUEDA DE ARVORE,PJ - PIRITUBA/JARAGUA,32027960,POINT (324233.088 7400496.72)


In [6]:
gdf_ocorrencias.shape

(4736, 5)

# Verificação de Geometrias

Apesar de a tabela parecer bem estruturada, não conseguimos plotar o mapa utilizando de um explore. Um dos possíveis motivos é que a tabela 'geometry' não esteja sendo reconhecida como geometrias, ou que as geometrias não sejam válidas. Vamos verificar isso a seguir

In [7]:
gdf_ocorrencias['verify_geom'] =  gdf_ocorrencias['geometry'].is_valid
gdf_ocorrencias.loc[gdf_ocorrencias['verify_geom'] ==False]

,data,ocorrencia,subpreit,name,geometry,verify_geom


In [8]:
gdf_ocorrencias.set_geometry(
    col='geometry', 
    inplace=True, 
    crs='EPSG:31983'
)

Como, aqui, já definimos o crs, não será necessário checar o crs do arquivo.

In [9]:
gdf_ocorrencias.sample(2)

,data,ocorrencia,subpreit,name,geometry,verify_geom
4040,2024-01-10,QUEDA DE ARVORE,PI - PINHEIROS,31910345,POINT (329747.08 7391696.465),True
4516,2024-01-23,QUEDA DE ARVORE,CS - CAPELA DO SOCORRO,32004001,POINT (329606.222 7372492.402),True


In [10]:
gdf_ocorrencias.explore()

Pronto, percebemos que a geomtria não estava setada corretamente e corrigimos o o problema.

# Padronizaçã de colunas

A coluna 'name' está claramente sendo usada como chave primária. Que, por padrão, tem 'cd' ou 'id' como identificador. 

Para confirmar se realmente é um identificador confiável, veremos se é uma linha com valores únicos:

In [11]:
gdf_ocorrencias['name'].is_unique

True

In [12]:
sum(gdf_ocorrencias['name'].isnull())

0

Aqui também estão usando 'subpreit' para representar as subprefeituras, mas por padrão estivemos usando 'nm_subpref', e por isso alteraremos para isso.

In [13]:
gdf_ocorrencias.rename(
    columns={
        'name' : 'id_ocorrencias',
        'subpreit' : 'nm_subpref'
    },
    inplace=True
)

In [14]:
gdf_ocorrencias.sample(2)

,data,ocorrencia,nm_subpref,id_ocorrencias,geometry,verify_geom
726,2024-09-03,QUEDA DE ARVORE,BT - BUTANTA,33308217,POINT (320962.306 7389420.488),True
4475,2024-01-22,DESLIZAMENTO,PA - PARELHEIROS,31990817,POINT (331756.995 7358847.301),True


Originalmente não teria nada para apagar, mas como fizemos a coluna verify_geom, que é desnecessária após o fim da verificação, iremos apagá-la.

In [15]:
gdf_ocorrencias.drop(
    columns = 'verify_geom',
    axis=1,
    inplace=True
)

In [16]:
gdf_ocorrencias.sample(2)

,data,ocorrencia,nm_subpref,id_ocorrencias,geometry
410,2024-01-08,QUEDA DE ARVORE,JA - JABAQUARA,31895632,POINT (331866.729 7385658.926)
301,2024-10-12,QUEDA DE ARVORE,SA - SANTO AMARO,33524036,POINT (328567.805 7381469.426)


# Salvar GDF

In [17]:
save_parquet_excel(
    gdf_ocorrencias, 
    fname= 'ocorrencias_2024',
    data_path = data_path,
    data_subpath = 'assets'
)